In [1]:
import pandas as pd
import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
data_dir = '/root/courses_rec/data/raw/skill_taxonomy'

# EDA

## skills

In [2]:
digital_skills = pd.read_csv(data_dir + '/digitalSkillsCollection_en.csv')
green_skills = pd.read_csv(data_dir +'/greenSkillsCollection_en.csv')
lang_skills = pd.read_csv(data_dir +'/languageSkillsCollection_en.csv')
transversal_skills = pd.read_csv(data_dir +'/transversalSkillsCollection_en.csv')
research_skills = pd.read_csv(data_dir +'/researchSkillsCollection_en.csv')

In [3]:
hierachy = pd.read_csv(data_dir + '/skillsHierarchy_en.csv')
skills = pd.read_csv(data_dir + '/skills_en.csv')
groups = pd.read_csv(data_dir + '/skillGroups_en.csv')
relations = pd.read_csv(data_dir + '/broaderRelationsSkillPillar_en.csv')
related_skills = pd.read_csv(data_dir +'/skillSkillRelations_en.csv')

In [4]:
len(skills)

13939

In [5]:
skills.head(2)

,conceptType,conceptUri,skillType,reuseLevel,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,scopeNote,definition,inScheme,description
0,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0005c151-5b5a...,skill/competence,sector-specific,manage musical staff,manage staff of music\ncoordinate duties of mu...,NaN,released,2023-11-30T15:53:37.136Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,Assign and manage staff tasks in areas such as...
1,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00064735-8fad...,skill/competence,occupation-specific,supervise correctional procedures,oversee prison procedures\nmanage correctional...,NaN,released,2023-11-30T15:04:00.689Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Supervise the operations of a correctional fac...


In [6]:
skills.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13939 entries, 0 to 13938
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   conceptType     13939 non-null  object
 1   conceptUri      13939 non-null  object
 2   skillType       13934 non-null  object
 3   reuseLevel      13934 non-null  object
 4   preferredLabel  13939 non-null  object
 5   altLabels       13591 non-null  object
 6   hiddenLabels    153 non-null    object
 7   status          13939 non-null  object
 8   modifiedDate    13939 non-null  object
 9   scopeNote       234 non-null    object
 10  definition      3 non-null      object
 11  inScheme        13939 non-null  object
 12  description     13939 non-null  object
dtypes: object(13)
memory usage: 1.4+ MB


In [21]:
skills.description[100]

'Place stone blocks or paving stones, which have been cut to the right size and thickness beforehand, to build stone walls and stairs, to lay pavement or to insert door and window frames.'

## hierachy

In [22]:
len(hierachy)

640

In [25]:
hierachy.head(5)

,Level 0 URI,Level 0 preferred term,Level 1 URI,Level 1 preferred term,Level 2 URI,Level 2 preferred term,Level 3 URI,Level 3 preferred term,Description,Scope note,Level 0 code,Level 1 code,Level 2 code,Level 3 code
0,http://data.europa.eu/esco/skill/e35a5936-091d...,language skills and knowledge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L,NaN,NaN,NaN
1,http://data.europa.eu/esco/skill/e35a5936-091d...,language skills and knowledge,http://data.europa.eu/esco/skill/43f425aa-f45d...,languages,NaN,NaN,NaN,NaN,"Ability to communicate through reading, writin...",NaN,L,L1,NaN,NaN
2,http://data.europa.eu/esco/skill/e35a5936-091d...,language skills and knowledge,http://data.europa.eu/esco/skill/e434e71a-f068...,classical languages,NaN,NaN,NaN,NaN,"All dead languages, no longer actively used, o...",Excludes:\n- all languages that are actively ...,L,L2,NaN,NaN
3,http://data.europa.eu/esco/skill/335228d2-297d...,skills,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN
4,http://data.europa.eu/esco/skill/335228d2-297d...,skills,http://data.europa.eu/esco/skill/03e0b95b-67d1...,handling and moving,NaN,NaN,NaN,NaN,"Sorting, arranging, moving, transforming, fabr...",Excludes:\n- Specific skills involving interac...,S,S6,NaN,NaN


In [24]:
hierachy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Level 0 URI             640 non-null    object
 1   Level 0 preferred term  640 non-null    object
 2   Level 1 URI             636 non-null    object
 3   Level 1 preferred term  636 non-null    object
 4   Level 2 URI             608 non-null    object
 5   Level 2 preferred term  608 non-null    object
 6   Level 3 URI             452 non-null    object
 7   Level 3 preferred term  452 non-null    object
 8   Description             507 non-null    object
 9   Scope note              324 non-null    object
 10  Level 0 code            640 non-null    object
 11  Level 1 code            636 non-null    object
 12  Level 2 code            608 non-null    object
 13  Level 3 code            452 non-null    object
dtypes: object(14)
memory usage: 70.1+ KB


## Group

In [26]:
groups.head(2)

,conceptType,conceptUri,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,scopeNote,inScheme,description,code
0,SkillGroup,http://data.europa.eu/esco/isced-f/00,generic programmes and qualifications,NaN,NaN,released,NaN,NaN,http://data.europa.eu/esco/concept-scheme/isce...,Generic programmes and qualifications are thos...,00
1,SkillGroup,http://data.europa.eu/esco/isced-f/000,generic programmes and qualifications not furt...,NaN,NaN,released,2023-07-13T15:16:44.968Z,NaN,http://data.europa.eu/esco/concept-scheme/skil...,NaN,000


In [27]:
groups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   conceptType     640 non-null    object
 1   conceptUri      640 non-null    object
 2   preferredLabel  640 non-null    object
 3   altLabels       112 non-null    object
 4   hiddenLabels    17 non-null     object
 5   status          640 non-null    object
 6   modifiedDate    341 non-null    object
 7   scopeNote       324 non-null    object
 8   inScheme        640 non-null    object
 9   description     507 non-null    object
 10  code            640 non-null    object
dtypes: object(11)
memory usage: 55.1+ KB


In [34]:
groups.description.nunique()

425

## Relations

In [35]:
relations.head(2)

,conceptType,conceptUri,broaderType,broaderUri
0,SkillGroup,http://data.europa.eu/esco/isced-f/00,SkillGroup,http://data.europa.eu/esco/skill/c46fcb45-5c14...
1,SkillGroup,http://data.europa.eu/esco/isced-f/000,SkillGroup,http://data.europa.eu/esco/isced-f/00


In [44]:
len(relations)

20822

In [43]:
broader_skill = relations.broaderUri[0]
groups[groups["conceptUri"] == broader_skill]

,conceptType,conceptUri,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,scopeNote,inScheme,description,code
537,SkillGroup,http://data.europa.eu/esco/skill/c46fcb45-5c14...,knowledge,knowledge,NaN,released,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,NaN,K


## Skill related

In [36]:
related_skills.head(2)

,originalSkillUri,originalSkillType,relationType,relatedSkillType,relatedSkillUri
0,http://data.europa.eu/esco/skill/00064735-8fad...,knowledge,optional,knowledge,http://data.europa.eu/esco/skill/d4a0744a-508b...
1,http://data.europa.eu/esco/skill/000bb1e4-89f0...,knowledge,optional,knowledge,http://data.europa.eu/esco/skill/b70ab677-5781...


In [38]:
related_skills.relationType.value_counts()

relationType
optional     5629
essential     189
Name: count, dtype: int64

In [5]:
len(digital_skills) + len(green_skills) + len(lang_skills) + len(transversal_skills) + len(research_skills)

2369

In [6]:
digital_skills.head()

,conceptType,conceptUri,preferredLabel,status,skillType,reuseLevel,altLabels,description,broaderConceptUri,broaderConceptPT
0,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/000f1d3d-220f...,Haskell,released,knowledge,sector-specific,Haskell techniques,The techniques and principles of software deve...,http://data.europa.eu/esco/skill/21d2f96d-35f7...,computer programming | software and applicatio...
1,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00c04e40-35ea...,incremental development,released,knowledge,sector-specific,gradual development,The incremental development model is a methodo...,http://data.europa.eu/esco/isced-f/0613 | http...,software and applications development and anal...
2,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/013441c1-1f13...,KDevelop,released,knowledge,sector-specific,kdevplatform | KDevelop platform,The computer program KDevelop is a suite of so...,http://data.europa.eu/esco/skill/925463a7-d51f...,integrated development environment software | ...
3,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0189f448-179e...,Absorb (learning management systems),released,knowledge,sector-specific,Absorb,The learning system Absorb is an e-learning pl...,http://data.europa.eu/esco/skill/6c80d53c-d8c9...,learning management systems | database and net...
4,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/01d269d9-b058...,develop blockchain innovative architectures,released,skill/competence,cross-sector,build innovative blockchain architectures,Develop new blockchain architectures that addr...,http://data.europa.eu/esco/skill/b590d4e5-7c62...,designing ict systems or applications


# Process 

In [7]:
skills = skills.drop(['status', 'modifiedDate', 'definition', 'scopeNote'], axis=1)

In [8]:
groups = groups.drop(['status', 'modifiedDate', 'scopeNote', 'code'], axis=1)

In [9]:
merged = pd.merge(skills, relations, how='left', on='conceptUri',)
merged = merged.drop_duplicates(subset='conceptUri')
merged.drop('conceptType_y', axis=1, inplace=True)

In [10]:
groups = groups[['conceptType', 'conceptUri', 'preferredLabel']]
groups = groups.rename({
    'conceptType': 'broaderType',
    'conceptUri': 'broaderUri',
    'preferredLabel': 'broaderPreferredLabel'
}, axis=1)

In [11]:
merged_2 = pd.merge(merged, groups, how='left', on='broaderUri')
merged_2 = merged_2.drop('hiddenLabels', axis=1)

In [12]:
merged_2.drop(['conceptType_x', 'skillType', 'reuseLevel', 'broaderType_x', 'broaderType_y'], axis=1, inplace=True)

In [13]:
merged_2.head()

,conceptUri,preferredLabel,altLabels,inScheme,description,broaderUri,broaderPreferredLabel
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,manage musical staff,manage staff of music\ncoordinate duties of mu...,http://data.europa.eu/esco/concept-scheme/skil...,Assign and manage staff tasks in areas such as...,http://data.europa.eu/esco/skill/aab9f44d-fe0e...,supervising a team or group
1,http://data.europa.eu/esco/skill/00064735-8fad...,supervise correctional procedures,oversee prison procedures\nmanage correctional...,http://data.europa.eu/esco/concept-scheme/memb...,Supervise the operations of a correctional fac...,http://data.europa.eu/esco/skill/87dff904-fb74...,directing operational activities
2,http://data.europa.eu/esco/skill/000709ed-2be5...,apply anti-oppressive practices,apply non-oppressive practices\napply an anti-...,http://data.europa.eu/esco/concept-scheme/skil...,"Identify oppression in societies, economies, c...",http://data.europa.eu/esco/skill/fe317a4c-e4e0...,protecting and enforcing
3,http://data.europa.eu/esco/skill/0007bdc2-dd15...,control compliance of railway vehicles regulat...,monitoring of compliance with railway vehicles...,http://data.europa.eu/esco/concept-scheme/skil...,"Inspect rolling stock, components and systems ...",http://data.europa.eu/esco/skill/35dc2a23-cdc3...,ensuring compliance with legislation
4,http://data.europa.eu/esco/skill/00090cc1-1f27...,identify available services,establish available services\ndetermine rehabi...,http://data.europa.eu/esco/concept-scheme/memb...,Identify the different services available for ...,http://data.europa.eu/esco/skill/29fbde52-e525...,assisting people to access services


In [14]:
def create_semantic_document(row):
    preferred_label = row['preferredLabel']
    alt_labels = row.get('altLabels', '')
    description = row['description']
    big_label = row.get('broaderPreferredLabel', '')

    semantic_document = f"{preferred_label.title()}: {description}"

    if pd.notnull(alt_labels):
        alt_names = ", ".join(alt_labels.split('\n'))
        semantic_document += f". Also known as {alt_names}"

    if pd.notnull(big_label):
        semantic_document += f". Belongs to {big_label}"

    return semantic_document
    
merged_2['semantic_document'] = merged_2.apply(create_semantic_document, axis=1)

In [15]:
index = faiss.IndexFlatIP(1024)
faiss_ids = []
semantic_embeddings = []

In [16]:
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B", device='cuda')

In [17]:
model.eval()
for i, row in tqdm(merged_2.iterrows(), total=len(merged_2), desc="Indexing documents"):
    doc_id = row['preferredLabel']  # or any unique ID
    semantic_text = row['semantic_document']

    # Embed
    vector = model.encode(semantic_text)  # embed_text returns [ [vector] ] so take [0]

    # Append to FAISS list
    semantic_embeddings.append(vector)
    faiss_ids.append(doc_id)

# Convert to numpy array
semantic_embeddings = np.array(semantic_embeddings).astype("float32")

# Normalize again ONLY if using IndexFlatIP and forgot to normalize earlier (you didn't)
# faiss.normalize_L2(semantic_embeddings)

# Add to index
index.add(semantic_embeddings)

Indexing documents: 100%|██████████| 13939/13939 [18:50<00:00, 12.33it/s] 


In [18]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B", trust_remote_code=True)

# sentences = [
#     "The weather is lovely today.",
#     "It's so sunny outside!",
#     "He drove to the stadium."
# ]
# embeddings = model.encode(sentences)

# similarities = model.similarity(embeddings, embeddings)
# print(similarities.shape)
# # [3, 3]